<a href="https://colab.research.google.com/github/luanwillianzh/.fonts/blob/master/NovelBin_Downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clique abaixo para comerçar o serviço!

In [ ]:
!pip3 install -q html2epub googletrans==3.1.0a0 bs4 html2text markdown2
import html2epub
import requests
from multiprocessing import Pool
from googletrans import Translator
from bs4 import BeautifulSoup as bs
import html2text
from markdown2 import markdown as md
import os
from google.colab import files
import re

h2t = html2text.HTML2Text()
h2t.ignore_images = True
open("opf.xml", "w").write(requests.get("https://raw.githubusercontent.com/luanwillianzh/novelbin/refs/heads/main/opf.xml").text)
translator = Translator()

def split(lst, size):
  return [ lst[i:i+size] for i in range(0, len(lst), size) ]

class Novel:
  def __init__(self, novel_id):
    self.novel_id = novel_id
    self.js = requests.get(f"https://novelbin.vercel.app/novel/{novel_id}").json()
    self.name = self.js["resultado"]["title"]
    self.desc = self.js["resultado"]["desc"]
    self.cover = self.js["resultado"]["cover"]
    self.chapters = [ i for i in self.js["resultado"]["chapters"] ]

  def baixar(self, chunk):
    for chapter_id in chunk:
      if not os.path.exists(f"{self.novel_id}/{chapter_id}.html"):
        r = requests.get(f"https://novelbin.com/b/{self.novel_id}/{chapter_id}").text
        h = bs(r)
        div = str(h.find_all("div", {"class":"chr-c"})[0])
        title = translator.translate(h2t.handle(str(h.find_all("span", {"class":"chr-text"})[0].text)), dest="pt").text.replace("<", "&lt;").replace(">", "&gt;")
        text = h2t.handle(div).split("\n\n")
        js = translator.translate(text, dest="pt")
        trad = "\n\n".join([ i.text for i in js ])
        epcontent = md(trad.replace("<", "&lt;").replace(">", "&gt;"))
        open(f"{self.novel_id}/{chapter_id}.html", "w").write(f'''<html><head><title>{title}</title></head><body><h1>{title}</h1>{epcontent}</body></html>''')
        print(chapter_id)
      else:
        pass

  def create_file(self):
    if not os.path.exists(self.novel_id):
      os.makedirs(self.novel_id)
    chunks = split(self.chapters, 10)
    with Pool(10) as p:
      p.map(self.baixar, chunks)
    epub = html2epub.Epub(self.name)
    for i in self.chapters:
      content = open(f"{self.novel_id}/{i}.html", "r").read()
      chapter = html2epub.create_chapter_from_string(content)
      epub.add_chapter(chapter)
      print(i)
    img_cover = requests.get(self.cover).content
    open(f"{epub.IMAGE_DIR}/cover.jpg", "wb").write(img_cover)
    epub.opf.template_file = os.path.abspath("opf.xml")
    epub.publisher = "Novelbin"
    epub.creator = "Novelbin"
    epub.language = "pt"
    epub.create_epub("Output")


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langsmith 0.2.10 requires httpx<1,>=0.

# Seleção da novel

In [ ]:
from ipywidgets import interact, interactive, fixed, HTML
import ipywidgets as widgets
from IPython.display import display

novel = Novel(input("Novel ID: ").replace("https://novelbin.com/b/", "").replace("https://novelbin.me/novel-book/", ""))
widgets.HTML(value=f"<h1>{novel.name}</h1><p><img src='{novel.cover}'></p><p>{novel.desc}</p>")

Novel ID: https://novelbin.com/b/reverend-insanity


HTML(value="<h1>Reverend Insanity</h1><p><img src='https://novelbin.me/media/novel/reverend-insanity.jpg'></p>…

In [ ]:
novel.create_file()
filename = re.sub(r'\W+ ', ' ', novel.name)
files.download(f"Output/{filename}.epub")


chapter-61
chapter-481
chapter-1
chapter-421
chapter-241
chapter-361
chapter-301
chapter-121
chapter-181
chapter-541
chapter-2
chapter-62
chapter-242
chapter-362
chapter-422
chapter-482
chapter-302
chapter-122
chapter-3
chapter-182
chapter-542
chapter-4
chapter-423
chapter-63
chapter-363
chapter-243
chapter-303
chapter-123
chapter-483
chapter-5
chapter-183
chapter-543
chapter-244
chapter-64
chapter-424
chapter-364
chapter-124
chapter-6
chapter-304
chapter-484
chapter-184
chapter-7
chapter-544
chapter-65
chapter-425
chapter-365
chapter-245
chapter-125
chapter-8
chapter-305
chapter-485
chapter-545
chapter-185
chapter-366
chapter-66
chapter-9
chapter-426
chapter-246
chapter-126
chapter-306
chapter-486
chapter-10
chapter-546
chapter-67
chapter-367
chapter-186
chapter-427
chapter-11
chapter-68
chapter-487
chapter-247
chapter-307
chapter-187
chapter-547
chapter-368
chapter-127
chapter-69
chapter-12
chapter-428
chapter-488
chapter-13
chapter-369
chapter-188
chapter-308
chapter-248
chapter-548

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print(f"{len(os.listdir(novel.novel_id))} de {len(novel.chapters)}")

2334 de 2334
